First, we set the context so that our notebook can execute the module.

In [1]:
import sys
sys.path.insert(1, '../')
print(sys.executable)
print(sys.version)
%matplotlib ipympl

/home/ms/anaconda3/bin/python
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


We run imports. `controller` has the controllers, `pendulum` has the system model, `simulate` is a wrapper for the simulation. `Visualizer` displays the pendulum animation.

In [2]:
import numpy as np
from pendulum import controller, pendulum, sim
from pendulum.viz import Visualizer
from IPython.display import HTML
import matplotlib.pyplot as plt

We set some parameters. State variables are given as a 4x1 array of $$[x, \dot{x}, \theta, \dot{\theta}]$$ 

Angles are given in radians, so we can see below that our $\theta$ has a slight offset, which makes the system unstable at first.

In [3]:
dt = 0.01 # simulation timestep
t_final = 5 # total simulation time, seconds
pend = pendulum.Pendulum(
    2.0, # large mass
    3.0, # small mass
    2.0, # length (meters)
    initial_state=np.array([0,0,-.1,0]) # initial state
)
# force function takes a time argument and returns a force. For now, let's say no external forces.
force_fn = lambda t: 0 

Now, let's create a controller. To start, we can make an empty controller with no control action inside. In this controller we have an `__init__` method for setting controller parameters, and a policy method for actually implementing the policy. The `policy` method takes the system state, time, and timestep as arguments and returns a scalar `action` (as well as a dict of data values, for passing additional data about controller calculations through to results). For now, the `data` dict is empty.

In [10]:
class NoController(controller.Controller):
    def __init__(self):
        pass
    
    def policy(self, state, t, dt, plot=False):
        return 0, {}

cont = NoController()

Next, we create the simulation object. This is the wrapper for the simulation.

In [11]:
simulation = sim.Simulation(dt, t_final, force_fn)

Here, we actually run the simulation by calling the `simulate` method of the simulation we created above. We pass the specific controller and pendulum into this method. This method returns the simulation data in a dataframe, `results`.

In [13]:
results = simulation.simulate(pend, cont)

100%|██████████| 501/501 [00:00<00:00, 1930.91it/s]


The results from the simulation has all of the simulation information in it. It's a `pandas` `DataFrame` so it has nice API compatibility with `matplotlib`, enabling us to easily plot the simulation information. Here, each state variable is stored under `state` in the multi-indexed dataframe, so we can plot each by iterating through the multi-index:

In [18]:
fig, ax = plt.subplots()
for s in results['state']:
    ax.plot(results[('state', s)], label=s)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Finally, we can create an animation of the pendulum by first creating a `Visualizer` object and then calling its `animate` method, which returns a matplotlib animation.

In [20]:
visualizer = Visualizer(results, pend, speed=2)
anim = visualizer.animate()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Matplotlib will typically attempt to display the animation we created above inline whenever we call `animate`. However, if that doesn't work for some reason (or, it can be slow on some systems) we can pre-render an HTML5 video with the results of our simulation:

In [21]:
HTML(anim.to_html5_video())